Cek jumlah beserta struktur datasets

In [4]:
import os

root_path = "D:\Pothole Vision - AI Road Damage Detection\dataset\RDD2022_all_countries"
def count_files(path):
    return len([f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]) if os.path.exists(path) else 0

summary = []

for country in os.listdir(root_path):
    country_path = os.path.join(root_path, country)
    if os.path.isdir(country_path):
        train_images = count_files(os.path.join(country_path, "train", "images"))
        test_images = count_files(os.path.join(country_path, "test", "images"))
        xml_files = count_files(os.path.join(country_path, "train", "annotations", "xmls"))
        summary.append(f"{country} -> Train Images: {train_images}, Test Images: {test_images}, Annotations: {xml_files}")

for line in summary:
    print(line)

China_Drone -> Train Images: 2401, Test Images: 0, Annotations: 2401
China_MotorBike -> Train Images: 1977, Test Images: 500, Annotations: 1977
Czech -> Train Images: 2829, Test Images: 709, Annotations: 2829
India -> Train Images: 7706, Test Images: 1959, Annotations: 7706
Japan -> Train Images: 10506, Test Images: 2627, Annotations: 10506
Norway -> Train Images: 8161, Test Images: 2040, Annotations: 8161
United_States -> Train Images: 4805, Test Images: 1200, Annotations: 4805


China_drone tidak punya folder test, abaikan?
Selanjutnya Saya akan split datset dari Train & Test menjadi Train, Test, & Val


Jika dataset DIGABUNG semua negara menjadi satu dataset besar:
Keuntungan:

Model akan lebih general karena belajar dari berbagai jenis jalan, cuaca, kamera.

Bisa membantu jika nanti digunakan di Indonesia yang belum punya data.

Jumlah data menjadi sangat besar (10.000++), sangat bagus untuk deep learning.

Kekurangan:

Bisa menyebabkan bias ke negara dengan data terbanyak (misalnya India, Japan).

Anotasi antar negara mungkin memiliki inkonsistensi kecil (labeling style, noise).

Rekomendasi untuk kasus ini:
Karena kamu akan pakai untuk Indonesia, tapi belum punya data lokal, maka:

Gabungkan semua negara → latih model global, supaya kuat terhadap variasi.

Simpan metadata negara asalnya → bisa dipakai untuk evaluasi per negara.

Nanti, jika ada data Indonesia, kamu bisa fine-tune model global ke data lokal.

In [49]:
import os
import xml.etree.ElementTree as ET
from collections import Counter

root_path = "dataset/RDD2022_all_countries"

country_folders = [
    "China_Drone", "China_MotorBike", "Czech",
    "India", "Japan", "Norway", "United_States"
]

all_labels = []

for country in country_folders:
    annotation_folder = os.path.join(root_path, country, "train", "annotations", "xmls")
    if os.path.exists(annotation_folder):
        for filename in os.listdir(annotation_folder):
            if filename.endswith('.xml'):
                file_path = os.path.join(annotation_folder, filename)
                try:
                    tree = ET.parse(file_path)
                    root = tree.getroot()
                    for obj in root.findall('object'):
                        label = obj.find('name').text.strip()
                        all_labels.append(label)
                except Exception as e:
                    print(f"❌ Error parsing {file_path}: {e}")

label_counts = Counter(all_labels)

print("Distribusi label di semua dataset mentah:")
for label, count in label_counts.items():
    print(f"{label}: {count}")

Distribusi label di semua dataset mentah:
D10: 11830
D00: 26016
D20: 10617
Repair: 1046
D40: 6544
Block crack: 3
D44: 5057
D01: 179
D11: 45
D50: 3581
D43: 793
D0w0: 1


In [54]:
import os
import xml.etree.ElementTree as ET

root_path = "dataset/RDD2022_all_countries"

def clean_annotations(root_path, target_label='D00'):
    removed_files = 0
    kept_files = 0
    for country in os.listdir(root_path):
        country_path = os.path.join(root_path, country)
        if not os.path.isdir(country_path):
            continue

        # Proses folder train annotations (ubah jika perlu val/test juga)
        annotations_dir = os.path.join(country_path, 'train', 'annotations', 'xmls')
        images_dir = os.path.join(country_path, 'train', 'images')

        if not os.path.exists(annotations_dir):
            print(f"Folder anotasi tidak ditemukan: {annotations_dir}, dilewati.")
            continue

        for xml_file in os.listdir(annotations_dir):
            if not xml_file.endswith('.xml'):
                continue

            xml_path = os.path.join(annotations_dir, xml_file)
            image_file = xml_file.replace('.xml', '.jpg')
            image_path = os.path.join(images_dir, image_file)

            tree = ET.parse(xml_path)
            root = tree.getroot()

            # Cari objek yang labelnya bukan target_label, hapus mereka
            objects = root.findall('object')
            removed_objs = 0
            for obj in objects:
                label = obj.find('name').text.strip()
                if label != target_label:
                    root.remove(obj)
                    removed_objs += 1

            # Cek apakah setelah penghapusan masih ada objek
            if len(root.findall('object')) == 0:
                # Hapus XML dan gambarnya
                os.remove(xml_path)
                if os.path.exists(image_path):
                    os.remove(image_path)
                removed_files += 1
            else:
                # Simpan ulang XML yang sudah dibersihkan
                tree.write(xml_path)
                kept_files += 1

    print(f"Proses selesai.")
    print(f"File XML dan gambar yang dihapus: {removed_files}")
    print(f"File XML yang dipertahankan: {kept_files}")

clean_annotations(root_path)

Proses selesai.
File XML dan gambar yang dihapus: 24837
File XML yang dipertahankan: 13548


In [1]:
import os
import random
import shutil
import pandas as pd

# Lokasi dataset dan output
root_path = "D:\\Pothole Vision - AI Road Damage Detection\\dataset\\RDD2022_all_countries"
output_path = "D:\\Pothole Vision - AI Road Damage Detection\\dataset-mix"
train_val_split = 0.8  # 80% untuk train, 20% untuk val

# Membuat direktori output
os.makedirs(os.path.join(output_path, 'train', 'images'), exist_ok=True)
os.makedirs(os.path.join(output_path, 'train', 'annotations'), exist_ok=True)
os.makedirs(os.path.join(output_path, 'val', 'images'), exist_ok=True)
os.makedirs(os.path.join(output_path, 'val', 'annotations'), exist_ok=True)
os.makedirs(os.path.join(output_path, 'test', 'images'), exist_ok=True)

# Metadata untuk csv
metadata = []

def copy_file(src_file, dest_file):
    shutil.copy(src_file, dest_file)

# Proses per negara
for country in os.listdir(root_path):
    country_path = os.path.join(root_path, country)
    if not os.path.isdir(country_path):
        continue

    images_path = os.path.join(country_path, 'train', 'images')
    annotations_path = os.path.join(country_path, 'train', 'annotations', 'xmls')

    if not os.path.exists(images_path) or not os.path.exists(annotations_path):
        continue

    image_files = sorted([f for f in os.listdir(images_path) if f.endswith('.jpg')])
    annotation_files = sorted([f for f in os.listdir(annotations_path) if f.endswith('.xml')])

    # Pastikan hanya file yang cocok (image dan XML) yang digunakan
    matched_files = []
    for image in image_files:
        basename = os.path.splitext(image)[0]
        if f"{basename}.xml" in annotation_files:
            matched_files.append((image, f"{basename}.xml"))

    for image, annotation in matched_files:
        src_image = os.path.join(images_path, image)
        src_annotation = os.path.join(annotations_path, annotation)

        split = 'train' if random.random() < train_val_split else 'val'
        dest_image = os.path.join(output_path, split, 'images', image)
        dest_annotation = os.path.join(output_path, split, 'annotations', annotation)

        copy_file(src_image, dest_image)
        copy_file(src_annotation, dest_annotation)

        metadata.append({'filename': image, 'country': country, 'split': split})

    # Test set (copy ke test/images tanpa anotasi)
    test_images_path = os.path.join(country_path, 'test', 'images')
    if os.path.exists(test_images_path):
        for test_img in os.listdir(test_images_path):
            if test_img.endswith('.jpg'):
                src_test_img = os.path.join(test_images_path, test_img)
                dest_test_img = os.path.join(output_path, 'test', 'images', test_img)
                copy_file(src_test_img, dest_test_img)

# Simpan metadata
metadata_df = pd.DataFrame(metadata)
metadata_df = metadata_df.sort_values(by=['split', 'country', 'filename'])
metadata_df.to_csv(os.path.join(output_path, 'metadata.csv'), index=False)

print("✅ Dataset berhasil digabung dan displit.")

✅ Dataset berhasil digabung dan displit.


In [3]:
import os
import pandas as pd
from tabulate import tabulate

root_path = "D:\\Pothole Vision - AI Road Damage Detection\\dataset-mix"

def count_files(path, ext):
    return len([f for f in os.listdir(path) if f.endswith(ext)]) if os.path.exists(path) else 0

# Data jumlah file berdasarkan split
data = {
    'Split': ['Train', 'Val', 'Test'],
    'Images (.jpg)': [
        count_files(os.path.join(root_path, 'train', 'images'), '.jpg'),
        count_files(os.path.join(root_path, 'val', 'images'), '.jpg'),
        count_files(os.path.join(root_path, 'test', 'images'), '.jpg')
    ],
    'Annotations (.xml)': [
        count_files(os.path.join(root_path, 'train', 'annotations'), '.xml'),
        count_files(os.path.join(root_path, 'val', 'annotations'), '.xml'),
        '-'  # Test tidak punya anotasi
    ]
}

df_split = pd.DataFrame(data)

# Membaca metadata.csv
metadata_path = os.path.join(root_path, 'metadata.csv')
if os.path.exists(metadata_path):
    metadata = pd.read_csv(metadata_path)
    split_counts = metadata['split'].value_counts().reset_index()
    split_counts.columns = ['Split', 'Count']

    country_counts = metadata['country'].value_counts().reset_index()
    country_counts.columns = ['Country', 'Count']

    # Tampilkan tabel
    print("=== Jumlah File per Split ===")
    print(tabulate(df_split, headers='keys', tablefmt='pretty', showindex=False))
    print("\n=== Distribusi Metadata per Split ===")
    print(tabulate(split_counts, headers='keys', tablefmt='pretty', showindex=False))
    print("\n=== Distribusi Metadata per Negara ===")
    print(tabulate(country_counts, headers='keys', tablefmt='pretty', showindex=False))
else:
    print("❌ File metadata.csv tidak ditemukan.")

=== Jumlah File per Split ===
+-------+---------------+--------------------+
| Split | Images (.jpg) | Annotations (.xml) |
+-------+---------------+--------------------+
| Train |     10734     |       10734        |
|  Val  |     2814      |        2814        |
| Test  |     9035      |         -          |
+-------+---------------+--------------------+

=== Distribusi Metadata per Split ===
+-------+-------+
| Split | Count |
+-------+-------+
| train | 10734 |
|  val  | 2814  |
+-------+-------+

=== Distribusi Metadata per Negara ===
+-----------------+-------+
|     Country     | Count |
+-----------------+-------+
|  United_States  | 3907  |
|      Japan      | 2797  |
|     Norway      | 2518  |
| China_MotorBike | 1419  |
|      India      | 1109  |
|   China_Drone   | 1034  |
|      Czech      |  764  |
+-----------------+-------+


In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.yaml')  # atau ganti ke 'yolov8s.yaml' jika mau versi sedikit lebih besar

model.train(
    data='rdd2022.yaml',
    epochs=20,
    imgsz=640,
    batch=16,
    name='yolov8_d00_only',
    project='runs/train'
)